In [ ]:
#train_df

#A_31 행만 추출
product = ['A_31']
A_31_df = train_df[train_df['PRODUCT_CODE'].isin(product)]

# T_31 행만 추출
product = ['T_31']
T_31_df = train_df[train_df['PRODUCT_CODE'].isin(product)]

In [ ]:
A_31_df = A_31_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'Y_Quality','LINE','PRODUCT_CODE','Y_Class'])
T_31_df = T_31_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'Y_Quality','LINE','PRODUCT_CODE','Y_Class'])

In [ ]:
#NaN 값 또는 각 열의 value가 동일 할 시 해당 feature제거

delete_list_A_31 = []

for i in range(1, len(A_31_df.columns)-1):
    values = []
    column = A_31_df[f'X_{i}']
    
    #X_i의 value list에 등록
    column_val = column.values
    values = column_val.tolist()

    not_0 = []
    
    ## 0이 아니면 not_0 에 추가
    for j in range(len(values)):
        if values[j] != 0:
            not_0.append(values[j])
            
    result = sum(values)
    result_0 = round((sum(not_0) + 0),4)
    #X_i의 value가 모두 0 또는 같으면 열 삭제 후, 삭제한 열 이름 delete_list에 추가
    if result == 0:
        A_31_df = A_31_df.drop(columns=[f'X_{i}'])
        delete_list_A_31.append(f'X_{i}')
    
    if result_0 != 0:
        if result_0/(len(not_0)) == not_0[0]:
            A_31_df = A_31_df.drop(columns=[f'X_{i}'])
            delete_list_A_31.append(f'X_{i}')  
    i + 1
print('Done.')

In [ ]:
#NaN 값 또는 각 열의 value가 동일 할 시 해당 feature제거


delete_list_T_31 = []

for i in range(1, len(T_31_df.columns)-1):
    values = []
    column = T_31_df[f'X_{i}']
    
    #X_i의 value list에 등록
    column_val = column.values
    values = column_val.tolist()

    not_0 = []
    
    ## 0이 아니면 not_0 에 추가
    for j in range(len(values)):
        if values[j] != 0:
            not_0.append(values[j])
            
    result = sum(values)
    result_0 = round((sum(not_0) + 0),4)
    #X_i의 value가 모두 0 또는 같으면 열 삭제 후, 삭제한 열 이름 delete_list에 추가
    if result == 0:
        T_31_df = T_31_df.drop(columns=[f'X_{i}'])
        delete_list_T_31.append(f'X_{i}')
    
    if result_0 != 0:
        if result_0/(len(not_0)) == not_0[0]:
            T_31_df = T_31_df.drop(columns=[f'X_{i}'])
            delete_list_T_31.append(f'X_{i}')  
    i + 1
print('Done.')

In [ ]:
#데이터셋을 랜덤하게 80%의 훈련셋과 20%의 테스트셋으로 분리
xA_train, xA_test, yA_train, yA_test = train_test_split(A_31_df[0:], A31_ytrain, test_size=0.2, shuffle=True, stratify= A31_ytrain, random_state=34)

dtrain = xgb.DMatrix(data=xA_train, label =yA_train)
dtest = xgb.DMatrix(data=xA_test, label=yA_test)

# max_depth = 3, 학습률은 0.1, 예제가 다중분류이므로 목적함수(objective)는 'objective' : 'multi:softmax'
# 오류함수의 평가성능지표는 logloss
# 부스팅 반복횟수는 400
# 조기중단을 위한 최소 반복횟수는 100

params = {'max_depth' : 3,
         'eta' : 0.1, 
         'objective' : 'multi:softmax',
         'eval_metric' : 'mlogloss',
         'early_stoppings' : 100 ,
         'num_class' : 3}

num_rounds = 400

# train 데이터 세트는 'train', evaluation(test) 데이터 세트는 'eval' 로 명기
wlist = [(dtrain, 'train'), (dtest,'eval')]
# 하이퍼 파라미터와 early stopping 파라미터를 train() 함수의 파라미터로 전달
xgb_model = xgb.train(params = params, dtrain=dtrain, num_boost_round=num_rounds, evals=wlist)

A31 = xgb.DMatrix(data=test_A_31_df)

pred_probs1 = xgb_model.predict(A31)

print(pred_probs1)